# Visualize Activations

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 


# Load Dataframe

In [ ]:
train_full = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv", dtype=str)
print(train_full.shape)

In [ ]:
train_full.head(10)

# Extract Images

In [ ]:
train_full.id = train_full.id + '.tif'

print(train_full.head())

In [ ]:
train_path = "../input/histopathologic-cancer-detection/train/"
print('Training Images:', len(os.listdir(train_path)))

In [ ]:
print('Training Images:', len(os.listdir('../input/histopathologic-cancer-detection/train/')))

for i in range(10):
  img = plt.imread('../input/histopathologic-cancer-detection/train/' + train_full.id[i])
  print('Images shape', img.shape)

# View Sample of Images

In [ ]:
sample = train_full.sample(n=16).reset_index()

plt.figure(figsize=(8,8))

for i, row in sample.iterrows():

    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{row.id}')    
    label = row.label

    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0, -5, f'Class {label}', color='k')
        
    plt.axis('off')

plt.tight_layout()
plt.show()

# Load Model

In [ ]:
cnn = load_model('../input/cancer-detection-final2/cancer_model_v12.h5')
cnn.summary()

# Filter Visualization Functions
The functions in the cell below can be used to create a single filter visualization.

In [ ]:
def compute_loss(input_image, layer, filter_index):
    feature_extractor = tf.keras.Model(inputs=cnn.inputs, outputs=layer.output)
    activation = feature_extractor(input_image)
    # We avoid border artifacts by only involving non-border pixels in the loss.
    filter_activation = activation[:, 2:-2, 2:-2, filter_index]
    return tf.reduce_mean(filter_activation)

def gradient_ascent_step(img, layer, filter_index, learning_rate):
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img, layer, filter_index)
    # Compute gradients.
    grads = tape.gradient(loss, img)
    # Normalize gradients.
    grads = tf.math.l2_normalize(grads)
    img += learning_rate * grads
    return loss, img
def initialize_image():
    img = tf.random.uniform((1, 96, 96, 3))
    # ResNet50V2 expects inputs in the range [-1, +1].
    # Here we scale our random inputs to [-0.125, +0.125]
    return (img - 0.5) * 0.25


def visualize_filter(layer, filter_index, steps, learning_rate):
    img = initialize_image()
    for iteration in range(steps):
        loss, img = gradient_ascent_step(img, layer, filter_index, learning_rate)

    # Decode the resulting input image
    img = deprocess_image(img[0].numpy())
    return loss, img

def deprocess_image(img):
    # Normalize array: center on 0., ensure variance is 0.15
    img -= img.mean()
    img /= img.std() + 1e-5
    img *= 0.15

    # Clip to [0, 1]
    img += 0.5
    img = np.clip(img, 0, 1)

    # Convert to RGB array
    img *= 255
    img = np.clip(img, 0, 255).astype("uint8")
    return img

The function in the cell below will create visualizations for all filters in a selected layer.

In [ ]:
def display_layer_filters(layer_name, steps=60, learning_rate=1):
    layer = cnn.get_layer(name=layer_name)
            
    n_filters = layer.filters
    n_cols = 8
    n_rows = n_filters // n_cols
    
    print(f'{layer_name} - {n_filters} filters')
    
    plt.figure(figsize=[2*n_cols, 2*n_rows])
    for i in range(n_filters):
        plt.subplot(n_rows, n_cols, i+1)
        loss, img = visualize_filter(layer, i, steps, learning_rate)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

# Visualizing Individual Laters

In [ ]:
display_layer_filters('conv2d')

In [ ]:
display_layer_filters('conv2d_1', steps=200)

In [ ]:
display_layer_filters('conv2d_2', steps=200, learning_rate=1)

In [ ]:
display_layer_filters('conv2d_3', steps=200, learning_rate=1)

# Display All Convolutional Filters

In [ ]:
for layer in cnn.layers:
    if 'conv' in layer.name:
        display_layer_filters(layer.name, steps=200)